In [2]:
import cPickle
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
with open('BOTCOMMENTS2.json') as f:
    bot_data = json.loads(f.readlines()[0])

with open('BigBotComments.json') as f:
    big_bot_data = json.loads(f.readlines()[0])
    
with open('user_results.json') as infile:
    result_dict = json.load(infile)

In [16]:
bot_names = bot_data.keys()
string_comments = {}
for bot in bot_names:
    comments = bot_data[bot]
    comment = ""
    for c in comments:
        comment += c['body']
    string_comments[bot] = comment
bot_data = string_comments

In [17]:
n_feats = 2000
doc_by_vocab = np.empty([len(bot_data), n_feats])


def build_vectorizer(max_features, stop_words, norm='l2'):
    return TfidfVectorizer(max_features = max_features, 
                                stop_words = stop_words,
                                max_df = 0.9,
                                min_df = 1,
                                norm = norm)

tfidf_vec = build_vectorizer(n_feats, "english")
doc_by_vocab = tfidf_vec.fit_transform([bot_data[d] for d in bot_data.keys()]).toarray()

In [18]:
cPickle.dump( bot_data, open( "bot_data.p", "wb" ) )
cPickle.dump( big_bot_data, open( "big_bot_data.p", "wb" ) )
cPickle.dump( result_dict, open( "user_results.p", "wb" ) )
cPickle.dump( tfidf_vec, open( "vectorizer.p", "wb") )